In [9]:
!pip install underthesea scikit-learn pandas matplotlib seaborn regex gdown

import pandas as pd
import numpy as np
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datasets import load_dataset
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [2]:
!gdown https://uploader.sh/kckui/data.zip

Downloading...
From: https://uploader.sh/kckui/data.zip
To: /content/data.zip
100% 324k/324k [00:00<00:00, 720kB/s]


In [3]:
!unzip data.zip

Archive:  data.zip
  inflating: README.txt              
  inflating: test/sentiments.txt     
  inflating: dev/topics.txt          
  inflating: dev/sentiments.txt      
  inflating: test/sents.txt          
  inflating: train/topics.txt        
  inflating: train/sents.txt         
  inflating: dev/sents.txt           
  inflating: test/topics.txt         
  inflating: train/sentiments.txt    


In [4]:
data_path = './'

"""
Đọc file sents.txt và sentiments.txt từ thư mục, sau đó gộp lại thành DataFrame.
"""
def load_data_from_folder(folder_name):

    # Đọc file sents.txt
    sent_path = os.path.join(data_path, folder_name, 'sents.txt')
    with open(sent_path, 'r', encoding='utf-8') as f:
        sentences = [line.strip() for line in f.readlines()]

    """
    Đọc sentiments.txt (file nhãn)
    0: Negative
    1: Neutral
    2: Positive
    """
    label_path = os.path.join(data_path, folder_name, 'sentiments.txt')
    with open(label_path, 'r', encoding='utf-8') as f:
        labels = [int(line.strip()) for line in f.readlines()]

    # Tạo DataFrame
    df = pd.DataFrame({
        'sentence': sentences,
        'sentiment': labels
    })

    return df

train_df = load_data_from_folder('train')
dev_df = load_data_from_folder('dev')
test_df = load_data_from_folder('test')

print(f"Số lượng tập Train: {len(train_df)}")
print(f"Số lượng tập Test: {len(test_df)}")

# Mapping label
label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
print(train_df.head())

Số lượng tập Train: 11426
Số lượng tập Test: 3166
                                            sentence  sentiment
0                          slide giáo trình đầy đủ .          2
1     nhiệt tình giảng dạy , gần gũi với sinh viên .          2
2               đi học đầy đủ full điểm chuyên cần .          0
3  chưa áp dụng công nghệ thông tin và các thiết ...          0
4  thầy giảng bài hay , có nhiều bài tập ví dụ ng...          2


In [5]:
# Mấy đoạn dùng regex phía dưới Gemini nó chỉ em :>
def text_preprocessing(text):
    # 1. Chuyển về chữ thường
    text = text.lower()

    """
    2. Giữ lại các từ khóa đặc biệt trong README (như colonsmile, dotdotdot...)
    và các chữ cái tiếng Việt, loại bỏ ký tự lạ.
    Regex này giữ lại chữ cái, số và dấu gạch dưới (để giữ từ ghép sau này)
    """
    text = re.sub(r'[^\w\s]', ' ', text)

    # 3. Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text).strip()

    # 4. Tokenization (Tách từ)
    # format="text" sẽ nối từ ghép bằng dấu gạch dưới (VD: sinh_viên)
    tokens = word_tokenize(text, format="text")

    return tokens

# Áp dụng cho tập train và test
train_df['text_clean'] = train_df['sentence'].apply(text_preprocessing)
test_df['text_clean'] = test_df['sentence'].apply(text_preprocessing)

print("Dữ liệu sau khi xử lý:")
print(train_df[['sentence', 'text_clean']].head())

Dữ liệu sau khi xử lý:
                                            sentence  \
0                          slide giáo trình đầy đủ .   
1     nhiệt tình giảng dạy , gần gũi với sinh viên .   
2               đi học đầy đủ full điểm chuyên cần .   
3  chưa áp dụng công nghệ thông tin và các thiết ...   
4  thầy giảng bài hay , có nhiều bài tập ví dụ ng...   

                                          text_clean  
0                            slide giáo_trình đầy_đủ  
1         nhiệt_tình giảng_dạy gần_gũi với sinh_viên  
2                 đi học đầy_đủ full_điểm chuyên cần  
3  chưa áp_dụng công_nghệ_thông_tin và các thiết_...  
4  thầy giảng bài hay có nhiều bài_tập ví_dụ ngay...  


In [12]:
# Em prompt :>


# Xây dựng Pipeline
model = Pipeline([
    # TF-IDF: Chuyển văn bản thành vector
    # ngram_range=(1,2): Lấy cả từ đơn và cụm 2 từ liền nhau
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), min_df=2)),

    # Mô hình SVM
    ('clf', LinearSVC(random_state=42, dual='auto'))

])


"""
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', MultinomialNB())
])
"""

# Huấn luyện mô hình trên tập Train
model.fit(train_df['text_clean'], train_df['sentiment'])
#print("Huấn luyện hoàn tất!")

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=2, ngram_range=(1, 2))),
                ('clf', LinearSVC(random_state=42))])

In [13]:
#Này prompt luôn ạ :>
# Dự đoán trên tập Test
y_pred = model.predict(test_df['text_clean'])

# Tính toán các chỉ số
acc = accuracy_score(test_df['sentiment'], y_pred)
f1 = f1_score(test_df['sentiment'], y_pred, average='macro')

print("-" * 30)
print(f"Accuracy: {acc:.4f}")
print(f"F1-Score (Macro): {f1:.4f}")
print("-" * 30)

# Báo cáo chi tiết theo từng nhãn (0, 1, 2)
print("Classification Report:")
print(classification_report(test_df['sentiment'], y_pred,
                            target_names=['Negative (0)', 'Neutral (1)', 'Positive (2)']))

------------------------------
Accuracy: 0.8955
F1-Score (Macro): 0.6983
------------------------------
Classification Report:
              precision    recall  f1-score   support

Negative (0)       0.88      0.95      0.91      1409
 Neutral (1)       0.57      0.17      0.26       167
Positive (2)       0.92      0.92      0.92      1590

    accuracy                           0.90      3166
   macro avg       0.79      0.68      0.70      3166
weighted avg       0.88      0.90      0.88      3166



In [14]:
# --- THỬ MÔ HÌNH 2: NAIVE BAYES ---
model_nb = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), min_df=2)),
    ('clf', MultinomialNB())
])

# Train mô hình 2
model_nb.fit(train_df['text_clean'], train_df['sentiment'])

# Đánh giá mô hình 2
print("KẾT QUẢ MÔ HÌNH NAIVE BAYES:")
y_pred_nb = model_nb.predict(test_df['text_clean'])
print(classification_report(test_df['sentiment'], y_pred_nb, target_names=['Negative', 'Neutral', 'Positive']))

KẾT QUẢ MÔ HÌNH NAIVE BAYES:
              precision    recall  f1-score   support

    Negative       0.82      0.96      0.88      1409
     Neutral       0.00      0.00      0.00       167
    Positive       0.92      0.88      0.90      1590

    accuracy                           0.87      3166
   macro avg       0.58      0.61      0.59      3166
weighted avg       0.83      0.87      0.85      3166



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
